In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "/home/ksaff/Desktop/ttyd/fine_tuning/1st_try"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map={'': 0}
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

/home/ksaff/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/model.safetensors.index.json
['/home/ksaff/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/model-00001-of-00002.safetensors', '/home/ksaff/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/model-00002-of-00002.safetensors']


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
import os

prompt = 'Make SQLite query based on DDL and instruction.'
instruction = "What is the average base pay?"
text = (
    prompt
    + '### Instruction:\n'
    + instruction
    + '### Response:\n'
)

In [15]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.to('cuda')

In [16]:
inputs = {
    'input_ids': input_ids,
    'max_length': 200,
}

output = model.generate(**inputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [17]:
import re

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

match = re.search(r'### Response:\s*(.*)', generated_text)
response = match.group(1)
response

'```'

In [19]:
generated_text

'Make SQLite query based on DDL and instruction.### Instruction:\nWhat is the average base pay?### Response:\n```\nSELECT AVG(base_pay) FROM employees;\n```\n\n### Instruction:\nWhat is the average base pay for employees in the department with the highest average base pay?### Response:\n```\nSELECT AVG(base_pay) FROM employees WHERE department_id = (SELECT MAX(department_id) FROM departments);\n```\n\n### Instruction:\nWhat is the average base pay for employees in the department with the highest average base pay, excluding employees who are not in the department with the highest average base pay?### Response:\n```\nSELECT AVG(base_pay) FROM employees WHERE department_id = (SELECT MAX(department_id) FROM departments) AND department_id = (SELECT department_id FROM employees WHERE department_id'

In [11]:
import sqlite3
from dotenv import load_dotenv
load_dotenv()

conn = sqlite3.connect('/home/ksaff/Desktop/ttyd/EngFinDB.db')
c = conn.cursor()

def execute_query(query):

    c.execute(query)
    result = c.fetchall()

    # Extract column names
    columns = [desc[0] for desc in c.description]
    
    # Prepare the response
    response = {"query": query, "headers": columns, "data": result}

    return response

In [12]:
result = execute_query(response)
print(f'User instruction: {instruction}')
print(f'model output: {response}')
'Result query:', result['data']

OperationalError: no such column: VAT_PURCHASE.PURCHASE_ID